In [1]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 2.1MB 17.3MB/s 
     |████████████████████████████████| 3.3MB 49.3MB/s 
     |████████████████████████████████| 901kB 51.5MB/s 
     |████████████████████████████████| 1.2MB 18.6MB/s 


In [2]:
from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
import keras
import re
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
import shutil


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer
import re
import string
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm_notebook

In [4]:
tweets = pd.read_csv('/content/drive/MyDrive/dataset/gender_classification.csv', encoding='latin-1')

In [5]:
tweets = tweets[tweets['gender'].notna()]
tweets = tweets[tweets['text'].notna()]


In [6]:
nltk.download('stopwords')
with_hashtage = re.compile('['+string.punctuation[:2]+string.punctuation[3:]+']')
without_hashtage = re.compile('['+string.punctuation+']')
def cleaning_result(text ,hashtage_rule):
    # Text cleaning rules
    with_hashtage = re.compile('['+string.punctuation[:2]+string.punctuation[3:]+']')
    without_hashtage = re.compile('['+string.punctuation+']')
    result =  [one_tweet_clean(w,hashtage_rule) for w in text]  # loop overall tweets
    # result =  [textStemmer(w) for w in result]
    return result

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
def one_tweet_clean(text , hashtage_rule):
    goodWord = ['against', 'up', 'down','off', 'over', 'under', 'each', 'few', 'more', 'most' ,'such', 'no', 'nor', 'not', 'don', "don't", 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't",'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
    STOPWORDS =  ([w for w in stopwords.words('english') if (w not in goodWord) ])# delete stopwords from text
    english = r'[^a-zA-Z0-9#]+'
    removeMentions = r'@[A-za-z0-9]+\s?'
    removeNumber = r'(^\d+|\s\d+)'
    removeLink = r'(http|ftp|https)://[A-za-z0-99./]+'
    removeEmails = r'[A-za-z0-9]+@[A-za-z0-9]+\.[A-za-z]+\s?'
    text = re.sub(removeEmails,r'@example.com',text)# remove Emails
    text = re.sub(removeLink,r'https' , text) # remove links
    text = re.sub(removeMentions,r'mention@mention',text)# remove mentions
    text = text.lower()# lowercase text
    text = re.sub(hashtage_rule,r' ' , text) # remove punctuation
    text =re.sub(removeNumber,r' ',text) # remove Numbers 
    text =re.sub(english,r' ',text) # remove Numbers 
    text =" ".join([w for w in text.split() if (not w in STOPWORDS and not w in string.punctuation ) ])# delete stopwords from text
    return text

In [8]:
tweets['clean_text'] =  cleaning_result(tweets['text'],with_hashtage)

In [9]:
ans = 0
for i in tweets['clean_text']:
  ans = max(ans,len(i.split()))
print(ans)

29


In [10]:
def get_sentence_with_filter(sentences,labels,number_of_words):
  sentence_data = []
  sentiment_data = []
  for sentence , sentiment in zip(sentences,labels):
    if len(sentence.split()) < number_of_words:
      continue
    sentence_data.append(sentence)
    sentiment_data.append(sentiment)
  return sentence_data , sentiment_data

In [11]:
sentences,labels = get_sentence_with_filter(tweets['clean_text'],tweets['gender'],4)

In [12]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=len(tweets['gender'].unique()))

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
input_ids=[]
attention_masks=[]
for sent in tqdm_notebook(sentences):
    bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =29,pad_to_max_length = True,return_attention_mask = True)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)
labels=np.array(labels)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [14]:
train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.2)

print('Train inp shape {} Val input shape {}\nTrain label shape {} Val label shape {}\nTrain attention mask shape {} Val attention mask shape {}'.format(train_inp.shape,val_inp.shape,train_label.shape,val_label.shape,train_mask.shape,val_mask.shape))

Train inp shape (28152, 29) Val input shape (7038, 29)
Train label shape (28152,) Val label shape (7038,)
Train attention mask shape (28152, 29) Val attention mask shape (7038, 29)


In [15]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)

bert_model.compile(loss=loss,optimizer=optimizer,metrics=[metric])

In [ ]:
history=bert_model.fit([train_inp,train_mask],train_label,batch_size=32,epochs=4,validation_data=([val_inp,val_mask],val_label))

Epoch 1/4
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr